In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re
import time

In [2]:
BASE_RSPP_URL = 'https://rspp.ru'
RSPP_URL_REGISTR = f'{BASE_RSPP_URL}/activity/social/registr/'
REPORT_TYPES = {'ОУР': 'Отчет по устойчивому развитию',
                'СО': 'Социальный отчет',
                'ИО': 'Интегрированный отчет',
                'ЭО': 'Экологический отчёт'}
RE_YEARS = re.compile('....')
FOLDER_TO_SAVE_REPORTS = 'reports'

In [3]:
def parse_td(company_report_row):
    report_info = [td.find('div') for td in company_report_row.findAll('td', class_='register-table__colclass')]
    company_name = report_info[0].find('span').text
    sector = report_info[1].text
    years_list = RE_YEARS.findall(report_info[2].text)
    years = ', '.join(years_list)
    report_tag_a = report_info[3].find('a')
    report_link, report_type = report_tag_a['href'], report_tag_a.text.strip()
    return {'компания': company_name,
            'сектор': sector,
            'год': years,
            'тип отчета': report_type,
            'ссылка на отчет': report_link}

In [4]:
def parse_rspp():
    rspp_response = requests.get(RSPP_URL_REGISTR)
    assert rspp_response.status_code < 400, 'Не получилось подключиться к сайту РСПП'
    soup = BeautifulSoup(rspp_response.text, 'lxml')
    company_report_filter = soup.find('div', class_='company-report-filter')
    report_table = company_report_filter.find('table', class_='table_scroll-mobile')
    headline, *company_reports_list = report_table.findAll('tr', class_='register-table__row')
    company_reports_dicts = [parse_td(company_report_td) for company_report_td in tqdm(company_reports_list)]
    return pd.DataFrame.from_dict(company_reports_dicts)

In [5]:
rspp_df = parse_rspp()
rspp_df.head()

100%|█████████████████████████████████████| 1388/1388 [00:00<00:00, 7675.72it/s]


,компания,сектор,год,тип отчета,ссылка на отчет
0,РУДН,"Образование, здравоохранение",2021,ОУР,/download/7033aa33d6bfc2f5e069f547a5b15109/
1,"ОАО ""МРСК Урала""",Энергетика,2021,ИО,/download/af503e07dd6b861d1ed3048c36868cc9/
2,"ПАО ""Россети Сибирь""",Энергетика,2021,ИО,/download/57becde4be827f45bedf2a46f58d793a/
3,"ПАО ""Россети Юг""",Энергетика,2021,ИО,/download/851fbaea09387885cefa38bd3b6838f3/
4,ПАО «Россети Ленэнерго»,Энергетика,2021,ИО,/download/2034b8c8e84e4bba78049bdd976fc122/


In [6]:
rspp_df.to_csv('rspp_reports.csv')

In [7]:
CHUNK_SIZE = 400
def download_report_pdf(link, filename, folder_to_save=FOLDER_TO_SAVE_REPORTS):
    download_url = f'{BASE_RSPP_URL}{link}'
    report_response = requests.get(download_url, stream=True)
    assert report_response.status_code == 200, 'Не получилось скачать отчет с сайта РСПП'
    filename = f'{folder_to_save}/{filename}'
    with open(filename, 'wb') as f:
        for chunk in report_response.iter_content(CHUNK_SIZE):
            f.write(chunk)

In [8]:
def download_reports_pdf(rspp_df):
    for index, report in tqdm(rspp_df.iterrows()):
        link = report['ссылка на отчет']
        company = report['компания'].replace('/', '_')
        filename = f'{company}_{index}.pdf'
        try:
            download_report_pdf(link, filename)
        except Exception as e:
            print(e, link)
        time.sleep(2)

In [9]:
download_reports_pdf(rspp_df)

1388it [1:35:36,  4.13s/it] 
